###Open file

In [1]:
import glob
import os
import csv
import zipfile
import StringIO


s_name = "/Users/ucaiado/Dropbox/NEUTRINO/ALGO/DATA/pisa2012.csv.zip"
s_dict = "/Users/ucaiado/Dropbox/NEUTRINO/ALGO/DATA/pisadict2012.csv"

# for name in glob.glob('C:/Projects/abase/*.zip'):
#     base = os.path.basename(name)
#     filename = os.path.splitext(base)[0]


#     datadirectory = 'C:/Projects/abase/'
#     dataFile = filename
#     archive = '.'.join([dataFile, 'zip'])
#     fullpath = ''.join([datadirectory, archive])
#     csv_file = '.'.join([dataFile, 'csv']) #all fixed


#     filehandle = open(fullpath, 'rb')
#     zfile = zipfile.ZipFile(filehandle)
#     data = StringIO.StringIO(zfile.read(csv_file)) #don't forget this line!
#     reader = csv.reader(data)

#     for row in reader:
#         print row

###Create a function to read the file

In [2]:
import pandas as pd

In [3]:
df_labels = pd.read_csv(s_dict)
df_labels.columns=["VAR","DESCRIPTION"]

In [4]:
df_labels.head(5)

,VAR,DESCRIPTION
0,CNT,Country code 3-character
1,SUBNATIO,Adjudicated sub-region code 7-digit code (3-di...
2,STRATUM,Stratum ID 7-character (cnt + region ID + orig...
3,OECD,OECD country
4,NC,National Centre 6-digit Code


In [5]:
import csv
import zipfile

def _getFields(l_fields, row):
    '''
    Return a dictionary with the data into the row, filtering 
    the fields passed 
    '''
    return {k: row.get(k, None) for k in l_fields}

def getData(s_file, l_fields):
    '''
    Return a list of dictionaries with the data desired
    '''
    with open(s_file, 'rb') as f1:
        zfile = zipfile.ZipFile(f1)
        l_rtn = []
        with zfile.open(zfile.filelist[0].filename) as f:
            reader = csv.DictReader(f)
            for idx, row in enumerate(reader):
                #count lines processed
                if idx%50000==0: print "estou na linha {}".format(idx)
                #get data desired
                l_rtn.append(_getFields(l_fields, row))
        print "estou na linha {}".format(idx)
        
    return l_rtn    

In [37]:
l_test = list(df_labels.head(5).VAR)
%time d_data = getData(s_name, l_test)

estou na linha 0
estou na linha 50000
estou na linha 100000
estou na linha 150000
estou na linha 200000
estou na linha 250000
estou na linha 300000
estou na linha 350000
estou na linha 400000
estou na linha 450000
estou na linha 485489
CPU times: user 2min 18s, sys: 723 ms, total: 2min 19s
Wall time: 2min 19s


###let's see the countries names

In [40]:
df_country =  pd.DataFrame(d_data)

In [44]:
df_country.head()

,CNT,NC,OECD,STRATUM,SUBNATIO
0,Albania,Albania,Non-OECD,ALB0006,0080000
1,Albania,Albania,Non-OECD,ALB0006,0080000
2,Albania,Albania,Non-OECD,ALB0006,0080000
3,Albania,Albania,Non-OECD,ALB0006,0080000
4,Albania,Albania,Non-OECD,ALB0006,0080000


In [76]:
b=[x.strip() for x in df_country.CNT.unique()]
a=[x.strip() for x in df_country.NC.unique()]

In [79]:
[x for  x in a if x not in b]

['United Kingdom (excl.Scotland)',
 'United Kingdom (Scotland)',
 'Republic of Korea',
 'China (Shanghai)',
 'Perm (Russian Federation)',
 'Viet Nam']

In [78]:
[x for  x in b if x not in a]

['United Kingdom',
 'Korea',
 'China-Shanghai',
 'Perm(Russian Federation)',
 'Florida (USA)',
 'Connecticut (USA)',
 'Massachusetts (USA)',
 'Vietnam']

###I like more the CNT names. I will make just some changes

In [6]:
l_test = ["CNT","SUBJECT"]
%time d_data = getData(s_name, l_test)

estou na linha 0
estou na linha 50000
estou na linha 100000
estou na linha 150000
estou na linha 200000
estou na linha 250000
estou na linha 300000
estou na linha 350000
estou na linha 400000
estou na linha 450000
estou na linha 485489
CPU times: user 2min 21s, sys: 794 ms, total: 2min 22s
Wall time: 2min 22s


In [7]:
df =  pd.DataFrame(d_data)
b=[x.strip() for x in df.CNT.unique()]

In [8]:
d_newNames = dict(zip(b,b))
d_newNames["United Arab Emirates"] = "UAE"
d_newNames["United Kingdom"] = "UK"
d_newNames["Serbia"] = "Serbia"
d_newNames["Korea"] = "South Korea"
d_newNames["Chinese Taipei"] = "Taiwan"
d_newNames["Slovak Republic"] = "Slovakia"
d_newNames["Russian Federation"] = "Russia"
d_newNames["Perm(Russian Federation)"] = "Russia"
d_newNames["Hong Kong-China"] = "Hong Kong"
d_newNames["China-Shanghai"] = "China"
d_newNames["Macau"] = "Macau"
d_newNames["Connecticut (USA)"] = "USA"
d_newNames["Florida (USA)"] = "USA"
d_newNames["Massachusetts (USA)"] = "USA"
d_newNames["United States of America"] = "USA"

In [9]:
df.CNT = df.CNT.apply(lambda x: d_newNames[x])

In [11]:
print "Number of Rows: {}".format(df.shape[0])
print "Number of Countries in the dataset: {}".format(len(df.CNT.unique()))

Number of Rows: 485490
Number of Countries in the dataset: 64


###Also, I would like to use Gini Index in my visualizations...maybe I could change the color of the countries by it

Here is how it would be done in R
<code>
    > repGINI = vars %>%
    +         group_by(CNT, SUBJECT) %>%
    +         summarise_each(funs(Gini( (./sum(.)) * SCORE)), 
    +                        contains('W_FSTR')) %>%
    +         melt(id=c('CNT', 'SUBJECT'), variable.name='REPNAME', 
    +              value.name='GINI')
</code>

Think better, I will calculate it using R.... I guess that it would be faster.

In [44]:
l_test = ["CNT","SUBNATIO","STRATUM","OECD","NC","SCHOOLID","STIDSTD","PV1MATH","PV2MATH","PV3MATH","PV4MATH","PV5MATH","PV1READ","PV2READ","PV3READ","PV4READ","PV5READ","PV1SCIE","PV2SCIE","PV3SCIE","PV4SCIE","PV5SCIE","W_FSTUWT"]
%time d_data = getData(s_name, l_test)

estou na linha 0
estou na linha 50000
estou na linha 100000
estou na linha 150000
estou na linha 200000
estou na linha 250000
estou na linha 300000
estou na linha 350000
estou na linha 400000
estou na linha 450000
estou na linha 485489
CPU times: user 2min 21s, sys: 3.59 s, total: 2min 25s
Wall time: 2min 25s


In [45]:
df =  pd.DataFrame(d_data)

In [43]:
df.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
CNT,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,...,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania
NC,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,...,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania,Albania
OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,...,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD,Non-OECD
PV1MATH,406.8469,486.1427,533.2684,412.2215,381.9209,396.3312,438.0823,576.5773,434.1097,463.6314,...,339.1572,430.0592,377.2472,386.7503,425.4635,326.6163,380.8303,406.8469,296.3935,368.7568
PV1READ,249.5762,406.2936,401.21,547.363,311.7707,378.2544,373.2503,567.7589,313.9153,454.5082,...,378.2592,369.1171,430.1229,344.2568,452.8401,307.5608,315.4245,400.3929,360.2236,424.772
PV1SCIE,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
PV2MATH,376.4683,464.3325,481.0796,498.6836,328.1742,354.2686,540.1231,516.5992,353.1002,441.0422,...,300.2103,442.5222,371.0157,343.9087,408.3268,349.2055,351.2307,379.584,350.1402,335.2625
PV2READ,254.342,349.8975,404.3872,481.4353,141.7883,366.3398,374.0446,465.1099,332.9787,420.3529,...,334.9542,355.484,426.9456,296.1401,367.0546,307.5608,336.0766,408.4124,364.1951,413.5448
PV2SCIE,408.84,471.5964,428.7952,481.574,275.774,358.2059,462.458,596.3633,317.0832,343.6591,...,326.3148,361.5628,437.0944,360.6304,478.7766,426.5573,364.3603,359.6979,383.9426,390.6565
PV3MATH,344.5319,453.4273,489.6479,415.3373,403.7311,372.1841,489.4921,532.9568,392.826,425.4635,...,315.7891,483.027,396.7207,391.4239,387.2955,353.8791,378.4935,397.4996,274.5833,290.8631


In [36]:
df.head()

,CNT,NC,OECD,PV1MATH,PV1READ,PV1SCIE,PV2MATH,PV2READ,PV2SCIE,PV3MATH,...,PV4READ,PV4SCIE,PV5MATH,PV5READ,PV5SCIE,SCHOOLID,STIDSTD,STRATUM,SUBNATIO,W_FSTUWT
0,Albania,Albania,Non-OECD,406.8469,249.5762,None,376.4683,254.342,408.84,344.5319,...,175.7053,367.8105,381.9209,218.5981,392.9877,0000001,00001,ALB0006,0080000,8.9096
1,Albania,Albania,Non-OECD,486.1427,406.2936,None,464.3325,349.8975,471.5964,453.4273,...,369.7553,443.6218,476.0165,396.7618,454.8116,0000001,00002,ALB0006,0080000,8.9096
2,Albania,Albania,Non-OECD,533.2684,401.21,None,481.0796,404.3872,428.7952,489.6479,...,431.3938,512.7191,533.2684,401.21,499.6643,0000001,00003,ALB0006,0080000,8.4871
3,Albania,Albania,Non-OECD,412.2215,547.363,None,498.6836,481.4353,481.574,415.3373,...,425.0393,474.1141,454.2842,471.9036,426.5573,0000001,00004,ALB0006,0080000,8.4871
4,Albania,Albania,Non-OECD,381.9209,311.7707,None,328.1742,141.7883,275.774,403.7311,...,272.8495,403.5248,395.1628,260.1405,422.1746,0000001,00005,ALB0006,0080000,8.4871


In [38]:
len(df.ix[:,['CNT', 'SUBNATIO', 'STRATUM', 'OECD', 'NC','SCHOOLID', 'STIDSTD', 'W_FSTUWT']].values)

485490

In [39]:
len(df.index)

485490

In [46]:
df_aux = df.copy()

In [48]:
df.index = [df.CNT, df.SUBNATIO, df.STRATUM, df.OECD, df.NC, df.SCHOOLID, df.STIDSTD, df.W_FSTUWT]

In [49]:
df.drop(['CNT', 'SUBNATIO', 'STRATUM', 'OECD', 'NC','SCHOOLID', 'STIDSTD', 'W_FSTUWT'], inplace=True, axis=1)

In [51]:
df_aux = df.stack()

In [52]:
df_aux.shape

(7282350,)

In [17]:
df.groupby(l_test).count()

array([None], dtype=object)

###Look at what there are inside the data